## Statistical methods for brain encoding

This tutorial is designed for the UCAS-AI2024 course titled "Language and Brain Data Processing," focusing on statistical methods for brain encoding. Throughout this tutorial, we will delve into the processes involved in brain encoding, particularly examining how syntax is encoded within the brain. Key areas covered will include methods to quantify syntactic complexity, aligning stimuli with BOLD signals using the hemodynamic response function (HRF), and correlating syntactic complexity with fMRI data through General Linear Models (GLM).

### syntactic complexity quantifications using node count

In [1]:
import numpy as np
import scipy.io as scio

def node_count_bu(in_file):
    """
    computing the bottom-up node count by counting 
    the number of right parenthesis behind each word
    """
    with open(in_file, 'r', encoding='utf-8') as rf:
        embed = []
        for line in rf:
            line = line.strip().split()
            i = 0
            while i < len(line):
                if line[i].encode().isalpha() and line[i+2] == ')':
                    count = 0
                    i += 2
                    while i<len(line) and line[i] == ')':
                        count += 1
                        i += 1
                    embed.append(count)
                else:
                    i += 1

    return np.array(embed).reshape(len(embed), 1)

def node_count_td(in_file):
    """
    computing the top-down node count by counting 
    the number of left parenthesis between each word
    and the nearest right parenthesis before it.
    """
    with open(in_file, 'r', encoding='utf-8') as rf:
        embed = []
        count = 0
        for line in rf:
            line = line.strip().split()
            i = 0
            while i < len(line):
                if line[i].encode().isalpha() and line[i+2] == ')':
                    embed.append(count)
                    count = 0
                    i += 2
                elif line[i] == '(':
                    count += 1
                i += 1
    return np.array(embed).reshape(len(embed), 1)

parsed_root = 'data/parsed_text/'
node_count_root = 'data/word_feature/'
files = ['story_1_zh.txt', 'story_1_en.txt']
for f in files:
    nc_bu = node_count_bu(parsed_root+f)
    scio.savemat(node_count_root+f[:-4]+'_bu.mat', {'data':nc_bu})
    nc_td = node_count_td(parsed_root+f)
    scio.savemat(node_count_root+f[:-4]+'_td.mat', {'data':nc_td})
    
print('computing node count... Done')

computing node count... Done


In [2]:
nc_bu, nc_bu.shape

(array([[ 1],
        [ 1],
        [ 1],
        ...,
        [14],
        [ 1],
        [ 5]]),
 (1086, 1))

### To align with fMRI data, the generated node count features should be convolved with HRF and downsampled to the same rate as fMRI.

In [3]:
import scipy.io as scio
import hdf5storage as hdf5
import numpy as np
from nilearn import glm
import h5py

zs = lambda v: (v-v.mean())/v.std()

def convolve_feature(nc_file, time_file, out_file, ref_length, hrf):
    data = scio.loadmat(nc_file)
    data = data['data']
    word_time = scio.loadmat(time_file)
    start_time = word_time['start']
    end_time = word_time['end']
    length = int(end_time[0][-1]*100)
    time_series = np.zeros([length])
    t = 0
    for j in range(length):
        if t == data.shape[0]:
            break
        if j == int(start_time[0][t]*100):
            for k in range(j, int(end_time[0][t]*100)):
                time_series[k] += data[t]
            while(j == int(start_time[0][t]*100)):
                t += 1
                if t == data.shape[0]:
                    break
    conv_series = np.convolve(hrf, time_series)
    conv_series = conv_series[:length]
    conv_series_ds = [conv_series[j] for j in range(0, length, 71)]
    conv_series_ds = np.array(conv_series_ds)
    # dump first 19 TRs because they are blank
    word_feature = zs(conv_series_ds[19:ref_length+19].reshape(ref_length,1))
    tmp = {'word_feature':word_feature.astype('float32')}
    hdf5.writes(tmp, out_file, matlab_compatible=True)
    
node_count_root = 'data/word_feature/'
time_root = 'data/time_feature/'
convolved_root = 'data/convolved_feature/'
fmri_root = 'data/fmri/'
files = ['story_1_zh', 'story_1_en']
hrf = glm.first_level.spm_hrf(0.71, 71)

for f in files:
    ref_length = h5py.File(fmri_root+f+'.mat')['fmri_response'].shape[0]
    convolve_feature(node_count_root+f+'_bu.mat', time_root+f+'.mat', convolved_root+f+'_bu.mat', ref_length, hrf)
    convolve_feature(node_count_root+f+'_td.mat', time_root+f+'.mat', convolved_root+f+'_td.mat', ref_length, hrf)
    
print('convolving node count... Done')

/Applications/anaconda3/lib/python3.8/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


convolving node count... Done


### fMRI voxel-wise encoding experiment

In [4]:
import scipy.io as scio
import numpy as np
import itertools as itools
from tqdm import tqdm
import random
import statsmodels.api as sm

zs = lambda v: (v-v.mean(0))/v.std(0)

class glm:
    """ perform voxel-wise encoding """
    def __init__(self, result_dir, block_shuffle=False):
        self.result_dir = result_dir
        self.block_shuffle = block_shuffle

    def run_glm(self, fmri, feature):
        nTR, n_vox = fmri.shape
        _, n_feature = feature.shape
        if self.block_shuffle:
            print('shuffling data with blocks...')
            inds = self.block_shuffle_inds(nTR)
            feature = feature[inds]
            fmri = fmri[inds]
        
        W = np.zeros([n_feature, n_vox])

        for i in tqdm(range(n_vox)):
            model = sm.OLS(fmri[:, i], feature)
            results = model.fit()
            W[:,i] = results.params

        savefile = self.result_dir+'glm_weight.mat'
        scio.savemat(savefile, {'weight':W})

    def block_shuffle_inds(self, nTR):
        allinds = range(nTR)
        blocklen = 100
        indblocks = list(zip(*[iter(allinds)]*blocklen))
        if nTR%blocklen != 0:
            indblocks.append(range(len(indblocks)*blocklen, nTR))
        random.shuffle(indblocks)
        return list(itools.chain(*indblocks))


In [5]:
import yaml
import h5py
import numpy as np

# load fmri
fmri = h5py.File('data/fmri/story_1_en.mat', 'r')['fmri_response'][()]

# load feature
feature = h5py.File('data/convolved_feature/story_1_en_bu.mat', 'r')['word_feature'][()].T

print(fmri.shape, feature.shape)

(445, 59412) (445, 1)


In [6]:
# 2. fit GLM model
result_dir = 'results/'
glm_model = glm(result_dir)
glm_model.run_glm(fmri, feature)

print('GLM model analysis... Done')

100%|███████████████████████████████████| 59412/59412 [00:17<00:00, 3337.20it/s]

GLM model analysis... Done
